In [1]:
#-------------------------------------------------------------------------------------------------------------------------------
# By Alexandra Lee (July 2018) 
#
# Apply saved model to new samples including:
#
# Encode samples from new condition using saved model
# Encode test set using saved model
# Decode estimated gene experssion after LSA
#-------------------------------------------------------------------------------------------------------------------------------
import os
import pandas as pd
import numpy as np
from keras.models import model_from_json, load_model
from keras import metrics, optimizers

randomState = 123
from numpy.random import seed
seed(randomState)

Using TensorFlow backend.


In [2]:
# load arguments
input_file = os.path.join(os.path.dirname(os.getcwd()), "encoded","oxygen_level", "estimated_test_t90_2layer_10latent_encoded.txt")
model_file = os.path.join(os.path.dirname(os.getcwd()), "models", "oxygen_level", "tybalt_2layer_10latent_decoder_model.h5")
weights_file = os.path.join(os.path.dirname(os.getcwd()), "models", "oxygen_level", "tybalt_2layer_10latent_decoder_weights.h5")

# output files
out_file = os.path.join(os.path.dirname(os.getcwd()), "output", "oxygen_level", "esimated_test_t90_2layer_10latent.txt")

In [3]:
# read in data
data = pd.read_table(input_file, header = 0, sep = '\t', index_col = 0)
data

,0,1,2,3,4,5,6,7,8,9
GSM1267087_HZI1950_Pae_G1a.CEL,0.0,2.8105,2.933027,1.275784,4.812092,0.0,2.047977,5.385044,4.673862,7.561147


In [4]:
# read in saved models

loaded_model = load_model(model_file)
# load weights into new model
loaded_model.load_weights(weights_file)

/home/alexandra/anaconda3/envs/Pa/lib/python3.5/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
# Use trained model to encode new data into SAME latent space
reconstructed = loaded_model.predict_on_batch(data)

reconstructed_df = pd.DataFrame(reconstructed, index=data.index)
reconstructed_df

,0,1,2,3,4,5,6,7,8,9,...,5539,5540,5541,5542,5543,5544,5545,5546,5547,5548
GSM1267087_HZI1950_Pae_G1a.CEL,0.686177,0.72357,0.437746,0.67743,0.439992,0.364563,0.50503,0.587372,0.641727,0.192211,...,0.35716,0.543123,0.601631,0.641289,0.669666,0.16932,0.427658,0.584807,0.765351,0.728943


In [6]:
# Save latent space representation
reconstructed_df.to_csv(out_file, sep='\t')